In [ ]:
Для этой работы был выбран новый датасет - globalAirQuality
https://www.kaggle.com/datasets/smeet888/global-air-quality-data15-days-hourly-50-cities?resource=download

In [222]:
# Инициализация

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, roc_auc_score
)

pd.set_option("display.max_columns", 15)
pd.set_option("display.width", 200)

columns = ["pm25", "pm10", "no2", "so2", "o3", "co", "aqi", "temperature", "humidity", "wind_speed"]

df = pd.read_csv("globalAirQuality.csv")
df.head()

,timestamp,country,city,latitude,longitude,pm25,pm10,no2,so2,o3,co,aqi,temperature,humidity,wind_speed
0,2025-11-04 18:25:17.554219,US,New York,40.713,-74.006,50.295,108.938,27.998,6.539,52.568,1.096,108,18.504,70.168,3.725
1,2025-11-04 19:25:17.554219,US,New York,40.713,-74.006,32.083,63.043,36.120,4.021,43.536,1.075,90,5.838,80.088,8.969
2,2025-11-04 20:25:17.554219,US,New York,40.713,-74.006,42.250,82.553,26.935,9.538,23.320,0.977,84,31.833,62.783,9.650
3,2025-11-04 21:25:17.554219,US,New York,40.713,-74.006,30.403,79.951,63.536,7.609,31.369,0.230,158,23.140,89.153,8.956
4,2025-11-04 22:25:17.554219,US,New York,40.713,-74.006,21.083,66.423,38.997,6.919,45.615,1.085,97,13.632,76.499,4.017


In [223]:
# В качестве целевого класса выступит aqi - индекс качества воздуха

def aqi_to_category(aqi):
    if aqi <= 50:
        return 0 
    elif aqi <= 100:
        return 1
    elif aqi <= 150:
        return 2
    else:
        return 3

df['aqi_category'] = df['aqi'].apply(aqi_to_category)

count = df['aqi_category'].value_counts().sort_index()
total = len(df)

print('Баланс классов\n'
     '0 - aqi <= 50 отлично\n'
     '1 - aqi <= 100 хорошо\n'
     '2 - aqi <= 150 потенциально вредно для здоровья\n'
     '3 - aqi > 150 вредно для здоровья\n')
print(f'0: {count[0]}, {100*count[0]/total:2.3f}%')
print(f'1: {count[1]}, {100*count[1]/total:2.3f}%')
print(f'2: {count[2]}, {100*count[2]/total:2.3f}%')
print(f'3: {count[3]}, {100*count[3]/total:2.3f}%')

Баланс классов
0 - aqi <= 50 отлично
1 - aqi <= 100 хорошо
2 - aqi <= 150 потенциально вредно для здоровья
3 - aqi > 150 вредно для здоровья

0: 171, 0.950%
1: 8108, 45.044%
2: 8874, 49.300%
3: 847, 4.706%


In [224]:
# Подготовка kNN и дерева решений

if 'aqi' in columns:
    columns.remove('aqi')
x = df[columns]
y = df['aqi_category']

xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=1, stratify=y
)

# Масштабирование для kNN
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)
xtest_scaled = scaler.transform(xtest)


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(xtrain_scaled, ytrain)
ypred_knn = knn.predict(xtest_scaled)
yscore_knn = knn.predict_proba(xtest_scaled)
roc_knn = roc_auc_score(ytest, yscore_knn, multi_class='ovr', average='macro')

dt = DecisionTreeClassifier(random_state=1, max_depth=6)
dt.fit(xtrain, ytrain)
ypred_dt = dt.predict(xtest)
yscore_dr = dt.predict_proba(xtest)
roc_dt = roc_auc_score(ytest, yscore_dr, multi_class='ovr', average='macro')

In [226]:
# Оценка и сравнение результатов классификации

def evaluate_model(ytest, ypred, roc, model):
    acc = accuracy_size = accuracy_score(ytest, ypred)
    prec = precision_score(ytest, ypred, average='macro', zero_division=0)
    rec = recall_score(ytest, ypred, average='macro', zero_division=0)
    f1 = f1_score(ytest, ypred, average='macro', zero_division=0)
    
    print(f'\t\033[32m{model}\033[0m')
    print(f'Accuracy: {acc:.4f}')
    print(f'Precision: {prec:.4f}')
    print(f'Recall: {rec:.4f}')
    print(f'F1-score: {f1:.4f}')
    if roc is not None:
        print(f'ROC: {roc:.4f}')
    else:
        print(f'ROC: N/A')
    print(classification_report(ytest, ypred, zero_division=0))

evaluate_model(ytest, ypred_knn, roc_knn, 'kNN')
evaluate_model(ytest, ypred_dt, roc_dt, 'Дерево решений')

	kNN
Accuracy: 0.8436
Precision: 0.8676
Recall: 0.5371
F1-score: 0.5880
ROC: 0.9083
              precision    recall  f1-score   support

           0       0.83      0.15      0.25        34
           1       0.83      0.92      0.87      1622
           2       0.85      0.85      0.85      1775
           3       0.95      0.24      0.38       169

    accuracy                           0.84      3600
   macro avg       0.87      0.54      0.59      3600
weighted avg       0.85      0.84      0.83      3600

	Дерево решений
Accuracy: 0.9992
Precision: 0.9996
Recall: 0.9838
F1-score: 0.9915
ROC: 0.9917
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        34
           1       1.00      1.00      1.00      1622
           2       1.00      1.00      1.00      1775
           3       1.00      0.99      1.00       169

    accuracy                           1.00      3600
   macro avg       1.00      0.98      0.99      3600
weight

In [ ]:
Сравнение моделей kNN и дерева решений показало, что дерево решений достигает
практически идеальных результатов (Accuracy = 0.9992, F1 = 0.9915, ROC = 0.9917).
Это объясняется тем, что категории AQI рассчитываются по чётким пороговым
значениям входных признаков, которые дерево успешно восстанавливает.

Целевой признак - это категории AQI, которые рассчитываются по фиксированным
порогам из тех же признаков, что подаются на вход (pm25, o3, no2 и др.).
Дерево решений идеально подходит для восстановления пороговых правил,
поэтому достигает почти идеального результата.
    
Модель kNN, напротив, демонстрирует низкую чувствительность к редким классам
(recall = 0.15 для класса 0, 0.24 для класса 3), что делает её непригодной
для задач мониторинга, где критически важно обнаруживать экстремальные в
плане редкости состояния.

kNN чувствителен к дисбалансу: когда 90% данных - классы 1–2, соседи
редких объектов почти всегда из мажоритарных классов, поэтому
модель не замечает редкие случаи.